In [1]:
# library for extraction of data
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import csv
from time import sleep
import time

In [3]:
# function to extract the tweets from twitter cards 
def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    
    tweet = (username, text)
    return tweet

In [4]:
# create instance of web driver and automate the scrapping
driver = "./chromedriver"
driver = webdriver.Chrome(driver)

# jump to twitter account (Ex. Obama)
driver.get('https://twitter.com/barackobama')

# disable the things for automation
options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

sleep(5)

driver.find_element_by_link_text('Tweets').click()

In [5]:
# Need to scoll down the webpage, data get extracted as we scroll down the page and finally generate inturrpt as stoping critera
data = []
tweet_ids = set()
flag = 0

while True:
    # page_cards = driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/section/div/div/div[5]/div/div/article/div/div/div/div[2]')
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')    
    
    for card in page_cards[-50:]: # target the latest 50 cards (stoping criteria)
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
                
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)

driver.close()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.88)


- As we finish the scrolling or reach to 50 cards, working of script is stop and generate inturrpt as error:

        StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
         (Session info: chrome=87.0.4280.88)
- But we have gatter the data so far and we can check with file by executing below code

In [6]:
# write a file with extracted tweets and users
with open('Twitter_data.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)